In [5]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione DISK()

In [106]:
function disk(radius=1., angle=2*pi)
    function disk0(shape=[36, 2])
        V, FV = simplexGrid(shape)
        V = [angle/shape[1] 0;0 radius/shape[2]]*V
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u)] end, W)...)
        W, FW = simplifyCells(V, FV)
        FW = [cell for cell in FW if length(cell)==3]
        return W, FW
    end
    return disk0
end

disk (generic function with 3 methods)

In [2]:
@btime W,CW = Lar.disk()();

  354.109 μs (4228 allocations: 389.56 KiB)


In [5]:
@code_warntype Lar.disk()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#disk0#268"{Float64,Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(36, 2)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


La funzione NON è type unstable dato che Body::Tuple{Array,Array{Array{Int64,1},1}} .

In [4]:
@benchmark Lar.disk()()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  353.436 μs …   9.845 ms  ┊ GC (min … max):  0.00% … 92.14%
 Time  (median):     366.290 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   437.634 μs ± 516.218 μs  ┊ GC (mean ± σ):  10.65% ±  8.54%

  ██▆▅▄▄▃▁▁▁                                                    ▂
  ████████████▇▇▇▇▇▇▇▇▇▆▆▇▆▆▆▆▆▇▆▆▆▆▇▆▆▇▆▆▇▆▆▆▅▆███▇▇▇▆▆▇▆▄▅▅▁▄ █
  353 μs        Histogram: log(frequency) by time        879 μs <

 Memory estimate: 389.50 KiB, allocs estimate: 4226.

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [7]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()   

2

In [10]:
function simplifyCells(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         append!(FW, [[Set(outcell)...]])
    end
    return hcat(W...),FW
end

simplifyCells (generic function with 1 method)

In [11]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

Funzioni di supporto iniziali sempre riportate per far girare i nuovi codici . Passaggio eseguito su tutti i notebook

## Evoluzione del codice tra parallelizzazione e refactoring .

Ho deciso di riportate tutta l'evoluzione del codice e le modifiche fatte nel corso del tempo per capire e confrontare tutti gli aspetti che migliorano o non migliorano il codice in termini di prestazioni , stabilità , velocità esecuzione ecc. 

In [102]:
function disk1(radius=1., angle=2*pi)
    function disk01(shape=[36, 2])
        V, FV = Lar.simplexGrid(shape)
        V = [angle/shape[1] 0;0 radius/shape[2]]*V
        
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
         for k=1:n
         W[k] = V[:,k] 
    end
        
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u)] end, W)...)
        W, FW = simplifyCells(V, FV)
        FW = [cell for cell in FW if length(cell)==3]
    
        return W, FW
    end
    return disk01
end

@btime W,CW = disk1()();

  360.564 μs (4334 allocations: 392.91 KiB)


Refactoring e modifica del ciclo for portato esternamente per poter utilizzare di seguito le macro @inbounds e @threads

In [20]:
function disk2(radius=1., angle=2*pi)
    function disk02(shape=[36, 2])
        V, FV = Lar.simplexGrid(shape)
        V = [angle/shape[1] 0;0 radius/shape[2]]*V
        
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u)] end, W)...)
        W, FW = simplifyCells(V, FV)
        FW = [cell for cell in FW if length(cell)==3] 
        return W, FW
    end
    return disk02
end

@btime W,CW = disk2()();

  269.480 μs (2641 allocations: 237.97 KiB)


Con aggiunta di threads velocità prestazioni aumentata e numero di allocazioni diminuite rispetto a precedente , .

In [21]:
function disk3(radius=1., angle=2*pi)
    function disk03(shape=[36, 2])
        V, FV = Lar.simplexGrid(shape)
        V = [angle/shape[1] 0;0 radius/shape[2]]*V
        
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u)] end, W)...)
        W, FW = simplifyCells(V, FV)
        filter!(cell -> length(cell)==3,FW)
        return W, FW
    end
    return disk03
end

@btime W,CW = disk3()();

  256.254 μs (2520 allocations: 225.47 KiB)


Aggiungendo filter! sostituendo la condizione dell'if . Prestazioni aumentate ancora .